# hisea use case workflow notebook

This notebook provides an example of running the hisea workflow.

## prerequisites
1. build the docker containers to download the data following instructions on <https://github.com/c-scale-community/use-case-hisea/tree/main/scripts/download>
2. build the docker container to preprocess the data following instructions on ... TODO

## download ERA5 data
Notes:
* path to my .cdsapirc file\
`/Users/backeb/.cdsapirc`
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`

* Other input parameters:\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [1]:
!docker run -v /Users/backeb/.cdsapirc:/root/.cdsapirc -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_era5.py --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

2022-07-18 13:27:12,047 INFO Welcome to the CDS
2022-07-18 13:27:12,048 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-18 13:27:12,212 INFO Request is completed
2022-07-18 13:27:12,212 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data4/adaptor.mars.internal-1658134213.684331-26016-15-abfc85a3-68fe-434a-a535-a6be6af199be.nc to /data/era5.nc (17.9K)
2022-07-18 13:27:12,585 INFO Download rate 48.2K/s


## download CMEMS data
Notes:
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
* Other input parameters:\
`--username bbackeberg`\
`--password iaTmwJ7D`\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [2]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_cmems_physics.py --username bbackeberg --password iaTmwJ7D --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

2022-07-18 13:27:18.704 [ INFO] Asynchronous mode set
2022-07-18 13:27:18.704 [ INFO] Authenticating user bbackeberg for service https://nrt.cmems-du.eu/motu-web/Motu
2022-07-18 13:27:19.311 [ INFO] Requesting file to download (this can take a while)...
2022-07-18 13:27:24.758 [ INFO] Authenticating user bbackeberg for service https://nrt.cmems-du.eu/motu-web/Motu
2022-07-18 13:27:30.826 [ INFO] The product is ready for download
2022-07-18 13:27:30.827 [ INFO] Downloading file (this can take a while)...
2022-07-18 13:27:30.990 [ INFO] File type: application/x-netcdf
2022-07-18 13:27:30.990 [ INFO] File size: 1.3 MB (1267528 B)
2022-07-18 13:27:30.991 [ INFO] Downloading file /data/cmems_thetao_bottomT_so_zos_uo_vo.nc
2022-07-18 13:27:31.038 [ INFO] -   1.3 MB (5.2%)
2022-07-18 13:27:31.043 [ INFO] -   1.3 MB (10.3%)
2022-07-18 13:27:31.046 [ INFO] -   1.3 MB (15.5%)
2022-07-18 13:27:31.056 [ INFO] -   1.3 MB (20.7%)
2022-07-18 13:27:31.062 [ INFO] -   1.3 MB (25.9%)
2022-07-18 13:27:31

In [1]:
ls -ltr ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718

total 2520
-rw-r--r--  1 user  staff    18376 Jul 18 15:27 era5.nc
-rw-r--r--  1 user  staff  1267528 Jul 18 15:27 cmems_thetao_bottomT_so_zos_uo_vo.nc
drwxr-xr-x  6 user  staff      192 Jul 18 15:40 preprocess_output/


## load .nc into memory

In [1]:
import xarray as xr
dname = '/Users/backeb/OneDrive - Stichting Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/'

cmemsds = xr.open_dataset(dname+'cmems_thetao_bottomT_so_zos_uo_vo.nc')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/backeb/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/cmems_thetao_bottomT_so_zos_uo_vo.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [2]:
whos

Variable   Type      Data/Info
------------------------------
os         module    <module 'os' from '/Users<...>da3/lib/python3.9/os.py'>
sys        module    <module 'sys' (built-in)>
xr         module    <module 'xarray' from '/U<...>ages/xarray/__init__.py'>


## preprocessing with coastserv

Notes
* to build the preprocessing container I did the following
    * in `use-case-hisea/scripts/preprocessing/tide_physical_chemical/` do:\
    `docker build --tag preprocessing .`
* to run the preprocessing in the docker container for `boundary.py` do:\
`docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output`\
the above has the following input parameters:
    * the directory from which to get the downloaded CMEMS data:\
    `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
    * the directory where all the model files are:\
    `/Users/backeb/Documents/REPOS/use-case-hisea/fm_model`
    * the directory where to dump the data after the preprocessing step:\
   `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output`\

In [14]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output

ERROR conda.cli.main_run:execute(41): `conda run python boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output` failed. (See above for error)
 
Model reference date is: 2015-01-01 00:00:00
 
running with options interp = True, simultaneous = True, steric = true
files for phyc: 0
files for phyc: 0
files for phyc: 0
files for phyc: 0
files for o2: 0
files for no3: 0
files for po4: 0
files for so: 1
files for thetao: 1
files for uo: 1
files for zos: 1
files for phyc: 0
files for phyc: 0
files for si: 0
suspected missing subs:
Opal
PON1
POP1
POC1
OXY
NO3
PO4
Green
Diat
Si
subs:
['Opal', 'PON1', 'POP1', 'POC1', 'OXY', 'NO3', 'PO4', 'salinity', 'temperature', 'uxuy', 'steric', 'Green', 'Diat', 'Si']
# of files available:
[0 0 0 0 0 0 0 1 1 1 1 0 0 0]
['salinity', 'temperature', 'uxuy', 'steric'] newsublist
substance: salinity
reading data...
['so'] printing the csub value
reading data file cmems_thetao_bottomT_so_zos_uo_vo.nc